In [1]:
import sys
sys.path.insert(1, 'Classes/')
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from multiprocessing import current_process

from ScoreGetter import ScoreGetter
from dataset_utils import checkIfEarlyMidEnd
from dataset_utils import encodeBoard
from dataset_utils import getColumns

In [2]:
'''
from PGNParser import parseFromPGN

data = parseFromPGN('Datasets/CCRL_games.pgn', 3000000)
df = pd.DataFrame(data, columns=['board'])
#df = df.sample(frac=0.33)
df = df.drop_duplicates(subset=['board'])
df.to_csv('Datasets/CCRL_fens.csv', index=False)

df.tail()
'''

"\nfrom PGNParser import parseFromPGN\n\ndata = parseFromPGN('Datasets/CCRL_games.pgn', 3000000)\ndf = pd.DataFrame(data, columns=['board'])\n#df = df.sample(frac=0.33)\ndf = df.drop_duplicates(subset=['board'])\ndf.to_csv('Datasets/CCRL_fens.csv', index=False)\n\ndf.tail()\n"

In [3]:
df = pd.read_csv('Datasets/CCRL_fens.csv')
boards = df['board'].values
df.tail()

,board
2937874,8/8/8/8/6K1/8/3k4/q4q2 w - - 2 74
2937875,8/8/8/8/7K/8/3k4/q4q2 b - - 3 74
2937876,8/8/5q2/8/7K/8/3k4/5q2 w - - 4 75
2937877,8/8/5q2/8/6K1/8/3k4/5q2 b - - 5 75
2937878,8/8/5q2/8/6K1/5q2/3k4/8 w - - 6 76


#### We load an engine to get a score from the positions.

In [4]:
score_getter = ScoreGetter('/home/gaetan/Téléchargements/stockfish/stockfish', 'eval', 'go depth 1')

#### We create our dataset by getting an equal number of start, middle and end game positions and encoding them.

In [5]:
batch_size = 1000000
nb_sample = 3
offset = 12

current = current_process()
pos = current._identity[0]-1 if len(current._identity) > 0 else 0
pbar = tqdm(total=batch_size*nb_sample, desc='Splitting and encoding', position=pos)


for i in range(nb_sample):
    print(i+1)
    data = []
    for j in range(i * batch_size, min(boards.shape[0], i * batch_size + batch_size)):
        board = boards[j]
        try:
            data.append(np.append(encodeBoard(board), score_getter.getScore(board)))
            pbar.update(1)
        except:
            continue
    df = pd.DataFrame(data, columns=np.append(getColumns(), 'cp (Stockfish 13)'))
    df.to_csv('Datasets/Stockfish/dataset' + str(offset + i + 1) + '.csv', index=False)
    
pbar.close()


Splitting and encoding:   0%|          | 1/3000000 [00:00<128:00:42,  6.51it/s]

1


Splitting and encoding:  33%|███▎      | 999433/3000000 [20:05<138:40:57,  4.01it/s]

2


Splitting and encoding:  67%|██████▋   | 1998658/3000000 [40:15<76:03:24,  3.66it/s] 

3


Splitting and encoding:  98%|█████████▊| 2935461/3000000 [59:11<01:18, 826.61it/s]  
